In [4]:
import subprocess
import pandas as pd
import numpy as np
import os
import sys
from tqdm import tqdm
import xlsxwriter


In [15]:
def send_mail(addr, file_path):
    #Send email
    cwd = os.listdir()
    ps_path = "C:\\Windows\\System32\\WindowsPowerShell\\v1.0\\powershell.exe"
    
    a = subprocess.Popen([ps_path, ".\"./function_script.ps1\";", "&sendMail {} {}".format(addr, file_path)])
    output, errors = a.communicate()
    return output, errors

def send_mail_noleads(addr):
    cwd = os.listdir()
    ps_path = "C:\\Windows\\System32\\WindowsPowerShell\\v1.0\\powershell.exe"
    a = subprocess.Popen([ps_path, ".\"./function_script.ps1\";", "&sendMail_noleads {}".format(addr)])
    output, errors = a.communicate()
    return output, errors

def generate_files_send_mail(names, eposter, svar_interesse, live = False):
    logg_file = open("logg_fil.txt", "w+")
    logg_file.write("--------------Følgende feil og mangler:--------- \n \n")
    i = 0
    j = 0
    
    try:
        email_query = [x for x in svar_interesse.columns if "epostadresse" in x.lower()][0]
        number_query = [x for x in svar_interesse.columns if "telefonnummer" in x.lower()][0]
    except IndexError:
        email_query = [x for x in svar_interesse.columns if "email" in x.lower()][0]
        number_query = [x for x in svar_interesse.columns if "phone number" in x.lower()][0]
        
    for name, email in zip(names, eposter):
        try:
            q_alt = [x for x in svar_interesse.columns if name.lower() in x.lower()][0] #finds query corresponding to association  
            
            df_interested = svar_interesse.loc[svar_interesse[q_alt] == 1]
             
            #pulls all respondents who wished to be contacted by a given association
            if not df_interested.size:
                logg_file.write("{} ingen interesserte respondenter :(  \n".format(name))
                if live:
                    output, errors = send_mail_noleads(email)
                else: 
                    output, errors = (None, None)
                    
                if errors != None:
                    logg_file.write("Feilmeldging {} registrert for foreningen {}".format(errors, name))
                j += 1
                continue
            
            export_dir = os.getcwd()+"\\excel_filer"
            file_path = export_dir+"\\{}.xlsx".format(name.replace(" ", "_").replace("/", "_"))
            #a.to_excel(file_path) Remnant of when a was a pandas series object (could contain duplicate entries)
            
            workbook = xlsxwriter.Workbook(file_path, {'nan_inf_to_errors': True})
            worksheet = workbook.add_worksheet()
            col = 0 
            
            [worksheet.write(row, col, data) for row, data in enumerate(df_interested[email_query])]
            [worksheet.write(row, col+1, data) for row, data in enumerate(df_interested[number_query])]
            workbook.close()
            
            if live:
                output, errors = send_mail(email, file_path)
            else:
                output, errors = (None, None)
                
            if errors != None:
                logg_file.write("Feilmeldging {} registrert for foreningen {}".format(errors, name))
            i += 1

        except IndexError:
            j += 1
            logg_file.write("{}  FATAL: Fant ikke foreningen i survey spm. Manuell sjekk kreves \n".format(name))

        workdone = (i+j) /( len(svar_interesse.columns) - 2)
        print("\rProgress: [{0:50s}] {1:.1f}%".format('#' * int(workdone * 50), workdone*100), end="", flush=True)
    
    logg_file.write("\n -------------------------------------------- \n")
    
    logg_file.write("\nNumber of files/emails generated: {}".format(i))
    logg_file.write("\n Number of errors: {}".format(j))
    logg_file.write("\n Number of files to generate: {}".format(len(svar_interesse.columns) -2))
    logg_file.close()



In [16]:

#Reading of info and survey exccel files
uio_filenames  = ["TheStudentAssociationDayUiO-registeryourwisheshere-3156979.xlsx", 
                  "ForeningsdagenUiO-registrerforeningsønsker-3156958.xlsx"]
hioa_filenames = ["ForeningsdagenHiOA-registrerforeningsønsker-3157016.xlsx", 
                  "TheStudentAssociationDayHiOA-registeryourwisheshere-3157019.xlsx"]

hioa_assos = pd.read_excel("SIO-#48466-v1-Påmelding_til_Foreningsdagen_HiOA_høsten_2017.xlsx") 
uio_assos = pd.read_excel("SIO-#48463-v1-Påmelding_til_Foreningsdagen_UiO_høsten_2017.xlsx")



for filename in hioa_filenames:
    print("\n {} : \n ".format(filename))
    svar_interesse = pd.read_excel(filename)

    names = hioa_assos["1: Foreningens navn:"]
    eposter = hioa_assos["2.2: Kontaktpersonens epostadresse:"]

    generate_files_send_mail(names, eposter, svar_interesse, live = False)
    print("\n ------------------------------------------------------------ \n")

for filename in uio_filenames:
    print("\n {} : \n ".format(filename))
    svar_interesse = pd.read_excel(filename)

    names = uio_assos["1: Foreningens navn:"]
    eposter = uio_assos["2.2: Kontaktpersonens epostadresse:"]

    generate_files_send_mail(names, eposter, svar_interesse, live = False)
    print(" \n ------------------------------------------------------------ \n")

print("Finished")



 ForeningsdagenHiOA-registrerforeningsønsker-3157016.xlsx : 
 
Progress: [##################################################] 100.0%
 ------------------------------------------------------------ 


 TheStudentAssociationDayHiOA-registeryourwisheshere-3157019.xlsx : 
 
Progress: [##################################################] 100.0%
 ------------------------------------------------------------ 


 TheStudentAssociationDayUiO-registeryourwisheshere-3156979.xlsx : 
 
Progress: [##################################################] 100.0% 
 ------------------------------------------------------------ 


 ForeningsdagenUiO-registrerforeningsønsker-3156958.xlsx : 
 
Progress: [##################################################] 100.0% 
 ------------------------------------------------------------ 

Finished
